In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# # Path to current directory
# path_to_current_dir = os.getcwd()

# def generate_and_save_data(num_files=100, sigma_range=(0.1, 0.8), mu_range=(0, 100), snr_range=(0.001, 1), directory=path_to_current_dir):
#     t = np.linspace(0,100,2000)

#     def signal(sigma, mu, t):
#         f = (5 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((t - mu)/sigma)**2)
#         return f

#     def noise(t):
#         Noise = (np.sin(1.0 * 2 * np.pi * t) + 0.5 * np.cos(3.0 * 2 * np.pi * t)) * np.random.randn(2000)
#         return Noise

#     for i in range(num_files):
#         sigma = np.random.uniform(low=sigma_range[0], high=sigma_range[1])
#         mu = np.random.uniform(low=mu_range[0], high=mu_range[1])
#         expected_signal = signal(sigma, mu, t)
#         Noise = noise(t)
#         Noise_signal = Noise + expected_signal
#         signal_to_noise_ratio = np.mean(expected_signal)/np.mean(Noise_signal)

#         label_noise = r'$S/N={%.2f}$' % (signal_to_noise_ratio)
#         label_signal = r'$\sigma$=%.2f, $\mu$=%.2f' % (sigma, mu)
        
#         fig, axs = plt.subplots(2, sharex=True, sharey=True, figsize=(16,9))
#         axs[0].plot(t, expected_signal, label = label_signal, color = 'r')
#         axs[0].set_title('Signal')
#         axs[0].legend(loc='upper right')
#         axs[1].plot(t, Noise_signal, label =label_noise, color = 'b')
#         axs[1].set_title('Corrupted Signal')
#         axs[1].legend(loc='upper right')
#         for axs in axs.flat:
#             axs.set(xlabel='time', ylabel='Amplitude')
#         plt.show()  # Show plots

#         # save data to csv
#         df = pd.DataFrame({'Time': t, 'Noise_Signal': Noise_signal, 'Signal': expected_signal})
#         filename = os.path.join(directory, f'noise_data_{i}.csv')
#         df.to_csv(filename, index=False)
#         print(f'Data saved to {filename}')

# # Usage:
# generate_and_save_data(num_files=100, directory= r"c:\Users\12031\Desktop\Grand\Github\ML_efield_recons\Data")

In [2]:
# =============================================================================
# MODULES

import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import pandas as pd
import os

# =============================================================================
# DATASET

def dataset(ts, ndat, sigma_range=(0.1, 0.8), mu_range=(0, 100), snr_range=(0.001, 1)):
    """Create dataset, case study: sin convolved with gaussian."""
    train_dataset = []
    for idat in range(ndat):
        # Parameters traces: random or fixed values
        sigma = np.random.uniform(low=sigma_range[0], high=sigma_range[1])
        mu = np.random.uniform(low=mu_range[0], high=mu_range[1])

        def signal(sigma, mu, t):
            f = (5 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((t - mu)/sigma)**2)
            return f

        def noise(t):
            Noise = (np.sin(1.0 * 2 * np.pi * t) + 0.5 * np.cos(3.0 * 2 * np.pi * t)) * np.random.randn(len(t))
            return Noise

        tssig = mu
        omega = 1.0 / sigma
        trace = np.sin(2. * np.pi * (ts - tssig) * omega) * np.exp(-(ts - tssig) ** 2 / (sigma ** 2))
        Noise = noise(ts)
        Noise_signal = Noise + trace
        train_dataset.append([Noise_signal])

    return torch.tensor(np.array(train_dataset), dtype=torch.float32)

# Time bins
nts = 1024
ts = np.arange(0, nts, 1)

ndat_train = 256
ndat_valid = 64
ndat_test = 10
train_ds_tensor = dataset(ts, ndat_train)
valid_ds_tensor = dataset(ts, ndat_valid)
test_ds_tensor = dataset(ts, ndat_test)

# This signal does not include noise
# This can be done during training using a dedicated function

# =============================================================================
# VISUALIZE FAKE SIGNAL

fig = plt.figure()
ax = plt.gca()
plt.subplots_adjust(left=0.13)
for i in range(3):
    plt.plot(ts, np.array(train_ds_tensor)[i, 0, :], ls='-', lw=2)
ax.tick_params(labelsize=14)
# plt.xlabel(r'Simulation number', fontsize=14)
# plt.ylabel(r'$\log_{10} (E)$', fontsize=14)
# ax.set_xlim([-10000,10000])
# ax.set_ylim([-10000,10000])
plt.show()

# =============================================================================
# DATALOADER

batch_size = 1
train_loader = torch.utils.data.DataLoader(train_ds_tensor,
                                           batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_ds_tensor,
                                           batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_ds_tensor,
                                          batch_size=1,
                                          shuffle=True)

: 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os

device="cpu"
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    device="cuda:0"
print(device)

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 32, 3, padding=1),
            nn.ReLU(True),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, 3, padding=1),
            nn.ReLU(True),
            nn.MaxPool1d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(True),
            nn.ConvTranspose1d(32, 1, 3, stride=2, padding=1, output_padding=1),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

def load_data(directory):
    data_list = []
    target_list = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            data = pd.read_csv(filepath)
            noise_signal = data['Noise_Signal'].values
            pure_signal = data['Signal'].values
            data_list.append(noise_signal)
            target_list.append(pure_signal)
    return torch.tensor(data_list, dtype=torch.float32).unsqueeze(1), torch.tensor(target_list, dtype=torch.float32).unsqueeze(1)

data, target = load_data(r'C:\Users\12031\Desktop\Grand\Github\ML_efield_recons\Data')
dataset = TensorDataset(data, target)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# mean = data.mean(dim=0, keepdim=True)
# std = data.std(dim=0, keepdim=True)
# data = (data - mean) / std
# target = (target - mean) / std  # normalize target in the same way

# Initialize the autoencoder, optimizer, and loss function
model = Autoencoder()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
loss_values = []

# Training loop
for epoch in range(300):
    epoch_loss = 0
    for batch_data, _ in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_data)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss = len(train_loader)  # Average loss over all batches
    loss_values.append(epoch_loss)  # Store the average loss for this epoch
    print(f'Epoch {epoch+1}/{1000}, Loss: {loss.item():.4f}')

for batch_data, _ in train_loader:
    break  # just get the first batch


# Plotting loss versus epoch
plt.figure(figsize=(10, 7))
plt.plot(range(0, 300), loss_values, marker='o', linestyle='-')
plt.title('Loss versus Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

# Pass the data through the autoencoder
with torch.no_grad():
    reconstructions = model(batch_data)

# Convert tensors to numpy arrays for plotting
batch_data = batch_data.squeeze().numpy()
reconstructions = reconstructions.squeeze().numpy()

# Plot the original and reconstructed signals
n = 5  # number of signals to plot
plt.figure(figsize=(10, 7))
for i in range(n):
    # Determine the common y-axis limits
    ymin = min(batch_data[i].min(), reconstructions[i].min())
    ymax = max(batch_data[i].max(), reconstructions[i].max())
    
    # Original signals
    plt.subplot(n, 2, i * 2 + 1)
    plt.plot(batch_data[i])
    plt.ylim([ymin, ymax])  # Set y-axis limits
    plt.title(f'Original Signal {i+1}')
    if i == 0:
        plt.legend(['Original'], loc='upper right')
    
    # Reconstructions
    plt.subplot(n, 2, i * 2 + 2)
    plt.plot(reconstructions[i])
    plt.ylim([ymin, ymax])  # Set y-axis limits
    plt.title(f'Reconstructed Signal {i+1}')
    if i == 0:
        plt.legend(['Reconstructed'], loc='upper right')

plt.tight_layout()
plt.show()


cuda:0


C:\Users\12031\AppData\Local\Temp\ipykernel_23672\996339992.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  return torch.tensor(data_list, dtype=torch.float32).unsqueeze(1), torch.tensor(target_list, dtype=torch.float32).unsqueeze(1)


Epoch 1/1000, Loss: 0.7970
Epoch 2/1000, Loss: 0.6690
Epoch 3/1000, Loss: 0.5950
Epoch 4/1000, Loss: 0.5922
Epoch 5/1000, Loss: 0.5015
Epoch 6/1000, Loss: 0.4912
Epoch 7/1000, Loss: 0.4305
Epoch 8/1000, Loss: 0.3899
Epoch 9/1000, Loss: 0.3336
Epoch 10/1000, Loss: 0.3089
Epoch 11/1000, Loss: 0.2798
Epoch 12/1000, Loss: 0.2529
Epoch 13/1000, Loss: 0.2350
Epoch 14/1000, Loss: 0.2063
Epoch 15/1000, Loss: 0.1844
Epoch 16/1000, Loss: 0.1757
Epoch 17/1000, Loss: 0.1626
Epoch 18/1000, Loss: 0.1577
Epoch 19/1000, Loss: 0.1440
Epoch 20/1000, Loss: 0.1286
Epoch 21/1000, Loss: 0.1290
Epoch 22/1000, Loss: 0.1134
Epoch 23/1000, Loss: 0.1142
Epoch 24/1000, Loss: 0.1077
Epoch 25/1000, Loss: 0.1052
Epoch 26/1000, Loss: 0.0967
Epoch 27/1000, Loss: 0.0906
Epoch 28/1000, Loss: 0.0898
Epoch 29/1000, Loss: 0.0913
Epoch 30/1000, Loss: 0.0856
Epoch 31/1000, Loss: 0.0797
Epoch 32/1000, Loss: 0.0775
Epoch 33/1000, Loss: 0.0828
Epoch 34/1000, Loss: 0.0819
Epoch 35/1000, Loss: 0.0770
Epoch 36/1000, Loss: 0.0713
E

: 